This notebook provides the absolute minimum code necessary for making a submission and is meant for copying into your own pipeline. If you're looking for a more detailed explanation of how the API works, take a look at [this notebook instead](https://www.kaggle.com/sohier/detailed-api-introduction).

In [ ]:
import gresearch_crypto
import pandas as pd
import numpy as np
import lightgbm as lgb
from numpy import arange
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from scipy.stats import pearsonr

In [ ]:
class CFG:
    nof_trees = 100

# Functions

In [ ]:
def process_data(df):
    return 0

# Data

In [ ]:
df_train = pd.read_csv('../input/g-research-crypto-forecasting/train.csv')

xte = pd.read_csv('../input/g-research-crypto-forecasting/example_test.csv')


In [ ]:
feature_cols = ['Asset_ID', 'Close', 'Count', 'High', 'Low', 'Open', 'VWAP',
       'Volume', 'timestamp']

In [ ]:
df_train.dropna(axis = 0, inplace = True)

# Model

In [ ]:
ytrain = df_train['Target'].copy()
xtrain = df_train[feature_cols].copy()

In [ ]:
train_size = int(0.9 * xtrain.shape[0])
x0, y0 = xtrain.iloc[:train_size],ytrain.iloc[:train_size]
x1, y1 = xtrain.iloc[train_size:],ytrain.iloc[train_size:]

In [ ]:
col_subset = ['High', 'Low', 'Close', 'Open', 'Count','Volume']

In [ ]:
model = ElasticNet()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
grid = dict()
grid['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
grid['l1_ratio'] = arange(0, 1, 0.01)
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

x0a = pd.get_dummies(x0['Asset_ID'])
x1a = pd.get_dummies(x1['Asset_ID'])
model.fit(x0[col_subset],y0)

In [ ]:
pred = model.predict(x1[col_subset])

In [ ]:
np.corrcoef(y1,pred)

# Submission

In [ ]:

env = gresearch_crypto.make_env()
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    xte = test_df[col_subset]
    sample_prediction_df['Target'] = model.predict(xte)
    env.predict(sample_prediction_df)